In [1]:
import pandas as pd
import json
import logging
import os

In [2]:
leagues = ["Bundesliga", "EPL", "La_Liga", "Ligue_1", "Serie_A", "Liga_Nos", "Eredivisie", "Jupiler_Pro_League"]

In [3]:
def load_json(folder, league):
    json_file = "{}/{}_2025_match_data.json".format(folder, league)
    with open(json_file, "r") as json_data:
        data = json.load(json_data)
    return data

In [4]:
data_dict = {}
for league in leagues:
    data_dict[league] = load_json("json_data", league)

In [5]:
import pandas as pd
import logging
import os

def json_to_df(data, league):
    logging.info("Traitement de la ligue : {}".format(league))
    print("Traitement de la ligue : {}".format(league))

    cols = ["game", "game_id", "score", "event_id", "period_id", "team_id", "player_id", "player_name", "type_id", "date", "minute", "second", "outcome", "start_x", "start_y", "end_x", "end_y", "qualifiers", "related_player_id", "touch", "shot", "goal", "type_name"]
    
    # Vérifier si le fichier CSV existe
    csv_file = "csv_data/{}_events.csv".format(league)
    df = pd.DataFrame(columns=cols)
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)

    # Récupérer les matchs déjà présents dans le DataFrame
    matches_in_df = set(df['game_id'])

    rows = []
    batch_size = 30
    match_count = len(data)
    batch_count = match_count // batch_size + (1 if match_count % batch_size != 0 else 0)

    for i, (match_key, match) in enumerate(data.items(), 1):
        try:
            game_id = match.get("matchId", None)
            # Vérifier si le match est déjà dans le DataFrame
            if game_id in matches_in_df:
                logging.info("Le match {} existe déjà dans le DataFrame. Passage au suivant.".format(game_id))
                continue

            logging.info("Traitement du match {}/{}".format(i, match_count))
            print("Traitement du match {}/{}".format(i, match_count))
            
            game = match_key.split("2025-")[1]
            try:
                playerIdNameDictionary = match["matchCentreData"].get("playerIdNameDictionary", {})
            except AttributeError as e:
                logging.error("Une erreur AttributeError est survenue pour le match {}: {}".format(game, e))
                continue

            date = match["matchCentreData"].get("startDate", None)
            score = match["matchCentreData"].get("score", None)
            
            for event in match["matchCentreData"].get("events", []):
                event_id = event.get("id", None)
                period_id = event["period"].get("value", None)
                team_id = event.get("teamId", None)
                player_id = event.get("playerId", None)
                player_name = playerIdNameDictionary.get(str(player_id), None)
                type_id = event.get("eventId", None)
                minute = event.get("minute", None)
                second = event.get("second", None)
                outcome = event.get("outcomeType", {}).get("value", None) == 1
                start_x = event.get("x", None)
                start_y = event.get("y", None)
                end_x = event.get("endX", None)
                end_y = event.get("endY", None)
                qualifiers = event.get("qualifiers", None)
                related_player_id = None
                touch = event.get("isTouch", None)
                shot = event.get("isShot", False)
                goal = event.get("isGoal", False)
                type_name = event["type"].get("displayName", None)

                rows.append([game, game_id, score, event_id, period_id, team_id, player_id, player_name, type_id, date, minute, second, outcome, start_x, start_y, end_x, end_y, qualifiers, related_player_id, touch, shot, goal, type_name])

            # Ajouter les données au DataFrame par batch
            if i % batch_size == 0 or i == match_count:
                df_new = pd.DataFrame(rows, columns=cols)
                df = pd.concat([df, df_new], ignore_index=True)
                rows = []
                # Sauvegarder le DataFrame dans le fichier CSV
                df.to_csv(csv_file, index=False)
        
        except AttributeError:
            continue

    logging.info("Traitement terminé pour la ligue : {}".format(league))
    print("Traitement terminé pour la ligue : {}".format(league))

    return df


In [6]:
events_data_dict = {}
for league in leagues:
    events_data_dict[league] = json_to_df(data_dict[league], league)

Traitement de la ligue : Bundesliga
Traitement du match 1/18
Traitement du match 2/18
Traitement du match 3/18
Traitement du match 4/18
Traitement du match 5/18
Traitement du match 6/18
Traitement du match 7/18
Traitement du match 8/18
Traitement du match 9/18
Traitement du match 10/18
Traitement du match 11/18
Traitement du match 12/18
Traitement du match 13/18
Traitement du match 14/18
Traitement du match 15/18
Traitement du match 16/18
Traitement du match 17/18
Traitement du match 18/18
Traitement terminé pour la ligue : Bundesliga
Traitement de la ligue : EPL
Traitement du match 1/30
Traitement du match 2/30
Traitement du match 3/30
Traitement du match 4/30
Traitement du match 5/30
Traitement du match 6/30
Traitement du match 7/30
Traitement du match 8/30
Traitement du match 9/30
Traitement du match 10/30
Traitement du match 11/30
Traitement du match 12/30
Traitement du match 13/30
Traitement du match 14/30
Traitement du match 15/30
Traitement du match 16/30
Traitement du match 17/

ERROR:root:Une erreur AttributeError est survenue pour le match Cercle-Bruges-Gent: 'NoneType' object has no attribute 'get'


Traitement terminé pour la ligue : Eredivisie
Traitement de la ligue : Jupiler_Pro_League
Traitement du match 1/48
Traitement du match 2/48
Traitement du match 3/48
Traitement du match 4/48
Traitement du match 5/48
Traitement du match 6/48
Traitement du match 7/48
Traitement du match 8/48
Traitement du match 9/48
Traitement du match 10/48
Traitement du match 11/48
Traitement du match 12/48
Traitement du match 13/48
Traitement du match 14/48
Traitement du match 15/48
Traitement du match 16/48
Traitement du match 17/48
Traitement du match 18/48
Traitement du match 19/48
Traitement du match 20/48
Traitement du match 21/48
Traitement du match 22/48
Traitement du match 23/48
Traitement du match 24/48
Traitement du match 25/48
Traitement du match 26/48
Traitement du match 27/48
Traitement du match 28/48
Traitement du match 29/48
Traitement du match 30/48


ERROR:root:Une erreur AttributeError est survenue pour le match Anderlecht-Genk: 'NoneType' object has no attribute 'get'


Traitement du match 31/48
Traitement du match 32/48
Traitement du match 33/48
Traitement du match 34/48
Traitement du match 35/48
Traitement du match 36/48
Traitement du match 37/48
Traitement du match 38/48
Traitement du match 39/48
Traitement du match 40/48
Traitement du match 41/48
Traitement du match 42/48
Traitement du match 43/48
Traitement du match 44/48
Traitement du match 45/48
Traitement du match 46/48
Traitement du match 47/48
Traitement du match 48/48
Traitement terminé pour la ligue : Jupiler_Pro_League


In [11]:
events_data_dict["Jupiler_Pro_League"] = json_to_df(data_dict["Jupiler_Pro_League"], "Jupiler_Pro_League")

Traitement de la ligue : Jupiler_Pro_League
Traitement du match 61/246
Traitement du match 62/246
Traitement du match 63/246
Traitement du match 64/246
Traitement du match 65/246
Traitement du match 66/246
Traitement du match 67/246
Traitement du match 68/246
Traitement du match 69/246
Traitement du match 70/246
Traitement du match 71/246
Traitement du match 72/246
Traitement du match 73/246
Traitement du match 75/246
Traitement du match 76/246
Traitement du match 77/246
Traitement du match 78/246
Traitement du match 79/246
Traitement du match 80/246
Traitement du match 81/246
Traitement du match 82/246
Traitement du match 83/246
Traitement du match 84/246
Traitement du match 85/246
Traitement du match 86/246
Traitement du match 87/246
Traitement du match 88/246
Traitement du match 89/246
Traitement du match 90/246
Traitement du match 91/246
Traitement du match 92/246
Traitement du match 93/246
Traitement du match 94/246
Traitement du match 95/246
Traitement du match 96/246
Traitement 

ERROR:root:Une erreur AttributeError est survenue pour le match OH-Leuven-RWD-Molenbeek: 'NoneType' object has no attribute 'get'


Traitement du match 104/246
Traitement du match 106/246
Traitement du match 107/246
Traitement du match 108/246
Traitement du match 109/246
Traitement du match 110/246
Traitement du match 111/246
Traitement du match 112/246
Traitement du match 113/246
Traitement du match 114/246
Traitement du match 115/246
Traitement du match 116/246
Traitement du match 117/246
Traitement du match 118/246
Traitement du match 119/246
Traitement du match 120/246
Traitement du match 121/246
Traitement du match 122/246
Traitement du match 123/246
Traitement du match 124/246
Traitement du match 125/246
Traitement du match 126/246
Traitement du match 127/246
Traitement du match 128/246
Traitement du match 129/246
Traitement du match 130/246
Traitement du match 131/246
Traitement du match 132/246
Traitement du match 133/246
Traitement du match 134/246
Traitement du match 135/246
Traitement du match 136/246
Traitement du match 137/246
Traitement du match 138/246
Traitement du match 139/246
Traitement du match 